In [1]:
!pip install wrds

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.7 MB 2.6 MB/s eta 0:00:02
   ----- ---------------------------------- 0.4/2.7 MB 4.4 MB/s eta 0:00:01
   --------- ------------------------------ 0.7/2.7 MB 4.6 MB/s eta 0:00:01
   --------------- ------------------------ 1.1/2.7 MB 5.6 MB/s eta 0:00:01
   --------------------- ------------------ 1.5/2.7 MB 6.1 MB/s eta 0:00:01
   ---------------------------- ----------- 2.0/2.7 MB 6.9 MB/s eta 0:00:01
   ------------------------------------- -- 2.5/2.7 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 7.5 MB/s eta 0:00:00


In [2]:
!pip install fredapi

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import wrds
import pandas as pd
from fredapi import Fred

wrds_db = wrds.Connection()

fred_api_key = "44711e700d59445b9b377da74bc9df3b"

fred = Fred(api_key=fred_api_key)


Enter your WRDS username [yimengs]: yimengs
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


pgpass file created at C:\Users\yimengs\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [4]:
# Take characteristics associated with fama-french factors - Size, B/M, Operating Profit, Asset Growth, Momentum (1-3 months - "randomly" chosen)
sql_query = f"""
    SELECT eom, ret_exc_lead1m, prc_local, me, be_me, ope_be, at_gr1, ret_3_1
    FROM contrib.global_factor
    WHERE common=1 and exch_main=1 and primary_sec=1 and obs_main=1 and
    excntry='USA' and eom > '1990-01-01' and prc_local>=1
"""

data = wrds_db.raw_sql(sql_query)
data = data.sort_values([ 'eom'])
data

,eom,ret_exc_lead1m,prc_local,me,be_me,ope_be,at_gr1,ret_3_1
152402,1990-01-31,0.01305,6.625,11.88525,0.362298,0.842545,0.04733,0.136364
324271,1990-01-31,-0.020311,17.25,162.84,0.458425,0.2871,0.085756,0.145846
224293,1990-01-31,-0.005818,8.0,30.0,<NA>,<NA>,<NA>,-0.179487
324402,1990-01-31,0.054609,105.5,5442.0065,0.093456,0.216141,0.187412,0.073661
224080,1990-01-31,0.088522,13.25,415.24175,<NA>,<NA>,0.037441,0.143646
...,...,...,...,...,...,...,...,...
58626,2025-02-28,<NA>,6.06,1683.7407,<NA>,-0.150997,-0.115983,<NA>
58627,2025-02-28,<NA>,3.39,458.47716,<NA>,0.021309,-0.032762,<NA>
58628,2025-02-28,<NA>,44.51,4107.47182,<NA>,<NA>,0.07111,<NA>
58618,2025-02-28,<NA>,11.05,94.44435,<NA>,-446.82732,-0.098065,<NA>


In [5]:
data_clean=data.dropna()

In [6]:
data_clean.shape

(1624491, 8)

In [7]:
import numpy as np

codes = {
    # Interest rates
    "TB3MS": "TB3MS",        # 3-Month T-Bill
    "GS1": "GS1",            # 1Y Treasury
    "GS5": "GS5",            # 5Y Treasury
    "GS10": "GS10",          # 10Y Treasury
    "GS30": "GS30",          # 30Y Treasury

    # Credit & spreads inputs
    "BAA": "BAA",            # Baa yield
    "AAA": "AAA",            # Aaa yield

    # Prices / inflation
    "CPI": "CPIAUCSL",       # CPI (headline)
    "CPICORE": "CPILFESL",   # Core CPI
    "PCEPI": "PCEPI",        # PCE price index

    # Real activity
    "INDPRO": "INDPRO",      # Industrial production
    "TCU": "TCU",            # Capacity utilization
    "RPI": "W875RX1",        # Real personal income

    # Labor market
    "UNRATE": "UNRATE",      # Unemployment rate
    "ICSA": "ICSA",          # Initial claims (weekly)

    # Money & policy
    "M2": "M2SL",            # M2 money stock
    "FEDFUNDS": "FEDFUNDS",  # Federal funds rate

    # Vol & commodities
    "VIX": "VIXCLS",         # VIX index (daily)
    "OIL": "DCOILWTICO",     # WTI oil price (daily)

    # Employment (replacing NAPM)
    "PAYEMS": "PAYEMS"       # Nonfarm payrolls
}


start_date = "1990-01-01"

raw = {}
for name, code in codes.items():
    s = fred.get_series(code, observation_start=start_date)
    raw[name] = s

macro_raw = pd.DataFrame(raw).sort_index()

macro_m = macro_raw.resample("M").last()

# Create derived macro features

# Term spread & default spread (in percentage points)
macro_m["term_spread"] = macro_m["GS10"] - macro_m["TB3MS"]
macro_m["def_spread"] = macro_m["BAA"] - macro_m["AAA"]

# Headline inflation (monthly log-diff of CPI)
macro_m["inflation"] = np.log(macro_m["CPI"]).diff()

# Core inflation
macro_m["core_inflation"] = np.log(macro_m["CPICORE"]).diff()

# Real short rate proxy: Fed funds minus inflation
macro_m["real_ff"] = macro_m["FEDFUNDS"] - (macro_m["inflation"] * 12)  # annualize inflation

# Real money growth: log-diff of M2
macro_m["m2_growth"] = np.log(macro_m["M2"]).diff()

# Industrial production growth
macro_m["indpro_growth"] = np.log(macro_m["INDPRO"]).diff()

# Real personal income growth
macro_m["rpi_growth"] = np.log(macro_m["RPI"]).diff()

# Payroll employment growth
macro_m["payems_growth"] = np.log(macro_m["PAYEMS"]).diff()

# Oil price growth
macro_m["oil_ret"] = np.log(macro_m["OIL"]).diff()

# 6. Drop initial NaNs from differences
macro_m = macro_m.dropna()

feature_cols = [
    # original levels
    "TB3MS", "GS1", "GS5", "GS10", "GS30",
    "BAA", "AAA",
    "UNRATE", "ICSA",
    "M2", "FEDFUNDS",
    "VIX",
    "OIL",
    "INDPRO", "TCU", "RPI", "PAYEMS",
    "CPI", "CPICORE", "PCEPI",

    # derived features
    "term_spread",
    "def_spread",
    "inflation",
    "core_inflation",
    "real_ff",
    "m2_growth",
    "indpro_growth",
    "rpi_growth",
    "payems_growth",
    "oil_ret"
]

macro_X = macro_m[feature_cols].dropna()


C:\Users\yimengs\AppData\Local\Temp\ipykernel_16896\275195837.py:54: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  macro_m = macro_raw.resample("M").last()


In [8]:
macro_X.shape

(427, 30)

In [9]:
macro_X = macro_X.reset_index().rename(columns={"index": "date"})


In [10]:
data_clean['eom'] = pd.to_datetime(data_clean['eom'])
print(data_clean['eom'].dtype)
data_clean = data_clean.drop('prc_local', axis=1)

datetime64[ns]


C:\Users\yimengs\AppData\Local\Temp\ipykernel_16896\1109376474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['eom'] = pd.to_datetime(data_clean['eom'])


In [11]:
data_full = data_clean.merge(macro_X, left_on='eom', right_on='date')
data_full = data_full.drop(columns=["date"])
print(data_full.shape)
data_full.head()


(1620666, 37)


,eom,ret_exc_lead1m,me,be_me,ope_be,at_gr1,ret_3_1,TB3MS,GS1,GS5,...,term_spread,def_spread,inflation,core_inflation,real_ff,m2_growth,indpro_growth,rpi_growth,payems_growth,oil_ret
0,1990-02-28,0.065049,20.447,0.626693,0.16599,0.479253,-0.176471,7.74,8.11,8.42,...,0.73,0.92,0.003914,0.004532,8.193033,0.003908,0.009041,0.003557,0.002195,-0.051548
1,1990-02-28,-0.028602,40.168125,0.68751,0.219076,0.196787,-0.076923,7.74,8.11,8.42,...,0.73,0.92,0.003914,0.004532,8.193033,0.003908,0.009041,0.003557,0.002195,-0.051548
2,1990-02-28,0.072644,5.634937,0.876319,0.104293,0.034325,0.078947,7.74,8.11,8.42,...,0.73,0.92,0.003914,0.004532,8.193033,0.003908,0.009041,0.003557,0.002195,-0.051548
3,1990-02-28,0.053698,5012.3595,0.29146,0.460676,0.050157,-0.051596,7.74,8.11,8.42,...,0.73,0.92,0.003914,0.004532,8.193033,0.003908,0.009041,0.003557,0.002195,-0.051548
4,1990-02-28,0.07129,175.705125,1.416902,0.010831,0.003274,-0.157895,7.74,8.11,8.42,...,0.73,0.92,0.003914,0.004532,8.193033,0.003908,0.009041,0.003557,0.002195,-0.051548


In [12]:
data_full.to_csv('data.csv.gz', compression="gzip", index=False)